In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [6]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 출력 파서 설정
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿
prompt = PromptTemplate(
    template=(
        "사용자가 관심 있는 분야인 \"{subject}\"에 대해 "
        "한국의 유명한 장소나 활동 5가지를 알려주세요.\n"
        "{format_instructions}"
    ),
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# 체인 생성
chain = prompt | model | output_parser

#  여기서 직접 값 지정
subject = "영화" 

# 체인 실행
result = chain.invoke({"subject": subject})

# 출력
print(f"한국의 '{subject}' 관련 추천:")
print(result)

# CSV 저장
csv_filename = f"data/{subject.strip().replace(' ', '_')}_recommendations.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow([f"{subject} 추천"])
    for item in result:
        writer.writerow([item])

print(f"'{csv_filename}' 파일로 저장 완료!")



한국의 '영화' 관련 추천:
['한국에서 영화 관련 유명한 장소나 활동 5가지는 다음과 같습니다: ', '부산 영화의 전당', '서울 영화진흥위원회', '한국 영화 자료관', '부천 국제 판타스틱 영화제', '칸 영화제 한국 특별전 ', '또는 ', '부산 영화의 전당', '서울 영화진흥위원회', '한국 영화 자료관', '영화 도시 부산 촬영지', '서울 국제 영화제']
'data/영화_recommendations.csv' 파일로 저장 완료!


In [10]:
from enum import Enum
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# 파서 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 정의
template = """
당신은 영화 리뷰를 분석하는 감정 분석 전문가입니다.
다음 영화 리뷰의 감정을 분석하여 반드시 아래 세 가지 중 하나로만 답변하세요.

리뷰: "{text}"

{format_instructions}

주의사항:
- 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요.
- 다른 설명 없이 하나의 단어만 출력하세요.
"""

prompt = ChatPromptTemplate.from_template(template).partial(format_instructions=format_instructions)

# LLM 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 선택: 잘못된 출력에 대비한 교정 파서
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

# 테스트 리뷰
reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

# 감정 분석 실행
for i, review in enumerate(reviews, 1):
    print(f"\n[{i}] 리뷰: {review}")
    prompt_value = prompt.format(text=review)
    response = model.invoke(prompt_value)
    
    try:
        sentiment = fixing_parser.parse(response.content)
    except Exception as e:
        sentiment = "분석 실패"
    
    print(f"→ 감정 분류: {sentiment}")



[1] 리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
→ 감정 분류: Sentiment.NEGATIVE

[2] 리뷰: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
→ 감정 분류: Sentiment.POSITIVE

[3] 리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
→ 감정 분류: Sentiment.NEUTRAL


In [5]:
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 API 키 로드

#  출력 구조 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이 (숫자)")
    major: str = Field(description="전공 학과명")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="학생의 목표")

#  파서 설정
parser = PydanticOutputParser(pydantic_object=StudentInfo)
format_instructions = parser.get_format_instructions()

#  모델 초기화
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # 또는 OpenAI 공식 API
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

#  LLM이 포맷 틀리게 출력하는 걸 보완하는 교정 파서
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

#  프롬프트 템플릿
template = """
당신은 텍스트에서 학생 정보를 구조화하는 전문가입니다.
아래 자기소개 문장에서 이름, 나이, 전공, 취미들, 목표를 각각 추출해 주세요.

자기소개:
{text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template).partial(format_instructions=format_instructions)

#  테스트 입력
text = "안녕하세요! 저는 김철수이고 42살입니다. 바이올린을 전공하고 있어요. 취미로는 게임하기, 영화보기, 골프치기을 좋아합니다. 앞으로 훌륭한 바이올리니스트가 되는 것이 목표입니다."

#  체인 실행
prompt_value = prompt.format(text=text)
response = model.invoke(prompt_value)
output = fixing_parser.parse(response.content)

#  결과 출력
print(" 학생 정보:")
print(f"name: {output.name}")
print(f"age: {output.age}")
print(f"major: {output.major}")
print(f"hobbies: {', '.join(output.hobbies)}")
print(f"goal: {output.goal}")

 학생 정보:
name: 김철수
age: 42
major: 바이올린
hobbies: 게임하기, 영화보기, 골프치기
goal: 훌륭한 바이올리니스트가 되는 것


In [4]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pprint import pprint

# 1. 출력 구조 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행지 이름 (예: 부산, 제주도)"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일)"),
    ResponseSchema(name="budget", description="총 예산 (예: 30만원, 50달러 등)"),
    ResponseSchema(name="rating", description="추천도 (1~5점 사이 정수)"),
    ResponseSchema(name="activities", description="주요 활동들을 리스트로")
]

# 2. 출력 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 3. 프롬프트 설정
template = """
다음 여행 후기에서 핵심 정보를 추출하세요.

여행 후기: {text}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 4. 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 5. 입력 텍스트
text = (
    "지난 주에 일본 후쿠오카로 2박 3일 여행을 다녀왔어요. "
    "총 90만원 정도 썼는데 모모치 해변에서 바다구경하고, 시장에서 사시미미 먹고, "
    "돈키호테도 구경했어요. 정말 만족스러운 여행이었습니다. "
    "5점 만점에 2점 정도 줄 수 있을 것 같아요."
)

# 6. 체인 실행
chain = prompt | model | parser
output = chain.invoke({"text": text})

# 7. 결과 출력
print(" 구조화된 여행 정보:")
pprint(output)


 구조화된 여행 정보:
{'activities': '모모치 해변 구경, 사시미 먹기, 돈키호테 구경',
 'budget': '90만원',
 'destination': '후쿠오카',
 'duration': '2박3일',
 'rating': '2'}
